Problem Statement for Submission

B. Write a program to randomly generate k-SAT problems. The program must accept values
for k, m the number of clauses in the formula, and n the number of variables. Each
clause of length k must contain distinct variables or their negation. Instances generated
by this algorithm belong to fixed clause length models of SAT and are known as uniform
random k-SAT problems.


C. Write programs to solve a set of uniform random 3-SAT problems for different combinations of m and n, and compare their performance. Try the Hill-Climbing, Beam-Search with beam widths 3 and 4, Variable-Neighborhood-Descent with 3 neighborhood functions. Use two different heuristic functions and compare them with respect to penetrance.


In [ ]:
# B. Write a program to randomly generate k-SAT problems. The program must accept values for k, m the number of clauses in the formula, and n the number of variables. Each clause of length k must contain distinct variables or their negation. Instances generated by this algorithm belong to fixed clause length models of SAT and are known as uniform random k-SAT problems.


import random

def generate_k_sat(k, m, n):
    # Generate m random clauses, each with k distinct variables (from n total variables)
    clauses = []
    for _ in range(m):
        variables = random.sample(range(1, n+1), k)  # pick k distinct variables
        clause = [v if random.choice([True, False]) else -v for v in variables]  # randomly negate
        clauses.append(clause)
    return clauses

def format_cnf(clauses):
    # Convert numeric representation into human-readable CNF form
    result = []
    for clause in clauses:
        parts = []
        for lit in clause:
            if lit > 0:
                parts.append(f"x{lit}")
            else:
                parts.append(f"¬x{abs(lit)}")
        result.append("(" + " ∨ ".join(parts) + ")")
    return " ∧ ".join(result)

if __name__ == "__main__":
    k = int(input("Enter clause length k: "))
    m = int(input("Enter number of clauses m: "))
    n = int(input("Enter number of variables n: "))

    cnf = generate_k_sat(k, m, n)
    print("Generated CNF formula:")
    print(format_cnf(cnf))


Generated CNF formula:
(x1 ∨ ¬x2 ∨ ¬x4) ∧ (¬x1 ∨ ¬x3 ∨ ¬x4) ∧ (¬x1 ∨ x4 ∨ ¬x2) ∧ (x2 ∨ ¬x1 ∨ ¬x3) ∧ (x3 ∨ x4 ∨ x1)


In [6]:
# C. Write programs to solve a set of uniform random 3-SAT problems for different combinations of m and n, and compare their performance. Try the Hill-Climbing, Beam-Search with beam widths 3 and 4, Variable-Neighborhood-Descent with 3 neighborhood functions. Use two different heuristic functions and compare them with respect to penetrance.


import random

def generate_3_sat(m: int, n: int):
    clauses = []
    for _ in range(m):
        vars = random.sample(range(1, n + 1), 3)
        clause = [v if random.choice([True, False]) else -v for v in vars]
        clauses.append(clause)
    return clauses

def evaluate(clauses, assignment):
    satisfied = 0
    for clause in clauses:
        if any((lit > 0 and assignment[abs(lit)]) or (lit < 0 and not assignment[abs(lit)]) for lit in clause):
            satisfied += 1
    return satisfied

def heuristic1(clauses, assignment):
    return evaluate(clauses, assignment)

def heuristic2(clauses, assignment):
    return evaluate(clauses, assignment) / len(clauses)

def random_assignment(n):
    return {i: random.choice([True, False]) for i in range(1, n + 1)}

# ---------------- Hill-Climbing ----------------
def hill_climbing(clauses, n, heuristic):
    current = random_assignment(n)
    best_score = heuristic(clauses, current)
    while True:
        neighbors = []
        for var in current:
            neighbor = current.copy()
            neighbor[var] = not neighbor[var]
            neighbors.append(neighbor)

        neighbor_scores = [heuristic(clauses, nb) for nb in neighbors]
        max_score = max(neighbor_scores)

        if max_score <= best_score:
            break
        else:
            best_score = max_score
            current = neighbors[neighbor_scores.index(max_score)]

    return best_score

# ---------------- Beam Search ----------------
def beam_search(clauses, n, heuristic, width):
    beam = [random_assignment(n) for _ in range(width)]
    best = 0
    while True:
        candidates = []
        for state in beam:
            for var in state:
                neighbor = state.copy()
                neighbor[var] = not neighbor[var]
                candidates.append(neighbor)

        scored = [(heuristic(clauses, c), c) for c in candidates]
        scored.sort(reverse=True, key=lambda x: x[0])
        new_beam = [s[1] for s in scored[:width]]

        if heuristic(clauses, new_beam[0]) <= best:
            break
        best = heuristic(clauses, new_beam[0])
        beam = new_beam

    return best

# ---------------- Variable Neighborhood Descent ----------------
def vnd(clauses, n, heuristic):
    current = random_assignment(n)
    k = 1
    while k <= 3:
        improved = False
        neighbors = []

        if k == 1:
            for var in current:
                neighbor = current.copy()
                neighbor[var] = not neighbor[var]
                neighbors.append(neighbor)
        elif k == 2:
            vars = list(current.keys())
            for i in range(len(vars)):
                for j in range(i+1, len(vars)):
                    neighbor = current.copy()
                    neighbor[vars[i]] = not neighbor[vars[i]]
                    neighbor[vars[j]] = not neighbor[vars[j]]
                    neighbors.append(neighbor)
        elif k == 3:
            vars = list(current.keys())
            for i in range(len(vars)-2):
                neighbor = current.copy()
                for j in range(3):
                    neighbor[vars[i+j]] = not neighbor[vars[i+j]]
                neighbors.append(neighbor)

        best_neighbor = max(neighbors, key=lambda x: heuristic(clauses, x))
        if heuristic(clauses, best_neighbor) > heuristic(clauses, current):
            current = best_neighbor
            k = 1
            improved = True
        else:
            k += 1

    return heuristic(clauses, current)

# ---------------- Main ----------------
if __name__ == "__main__":
    m = int(input("Enter number of clauses (m): "))
    n = int(input("Enter number of variables (n): "))
    clauses = generate_3_sat(m, n)

    print("Hill-Climbing (h1):", hill_climbing(clauses, n, heuristic1))
    print("Hill-Climbing (h2):", hill_climbing(clauses, n, heuristic2))

    print("Beam Search width=3 (h1):", beam_search(clauses, n, heuristic1, 3))
    print("Beam Search width=4 (h1):", beam_search(clauses, n, heuristic1, 4))

    print("VND (h1):", vnd(clauses, n, heuristic1))
    print("VND (h2):", vnd(clauses, n, heuristic2))


Hill-Climbing (h1): 4
Hill-Climbing (h2): 1.0
Beam Search width=3 (h1): 4
Beam Search width=4 (h1): 4
VND (h1): 4
VND (h2): 1.0
